## 原始方法

In [7]:
import time
from py2neo import Graph
graph = Graph("http://localhost:7474", auth=("neo4j", "990910lhylhy"))

In [8]:
def get_node_edges_1(kword):
    if kword:
        c1 = 'MATCH (a{entityName:\'' + kword + '\'})-[r]->(n) RETURN a,n,r'
        start = time.time()
        ### time start
        s1 = graph.run(c1).data()# one hop
        # s2 = graph.run(c2).data()# two hop
        print(s1)
        nodes = list(map(buildSelf, s1)) + list(map(buildNodes, s1))
                # + list(map(buildNodes, s2))
        edges = list(map(buildEdges, s1))
                # + list(map(buildEdges, s2))
        elements = {"nodes": nodes, "edges": edges}  #为什么要套两层？？？？？？？？？？
        ### time end
        mid = time.time()
        print(mid - start) ### print time
#     return elements  #这里可以直接return nodes和edges,在get_initial graph 里调用

In [9]:
def buildSelf(nodeRecord):
    data = {"id": nodeRecord['a'].identity, "label": list(nodeRecord['a'].labels)[0],
            "name": nodeRecord['a']['entityName'], "entityId": nodeRecord['a']['entityId'],
            "entityProperty": nodeRecord['a']['entityProperty'], 
            "entityImportance":nodeRecord['a']['entityImportance']}
    return {"data": data}


def buildNodes(nodeRecord):
    data = {"id": nodeRecord['n'].identity, "label": list(nodeRecord['n'].labels)[0],
            "name": nodeRecord['n']['entityName'], "entityId": nodeRecord['n']['entityId'],
            "entityProperty": nodeRecord['n']['entityProperty'],
           "entityImportance":nodeRecord['n']['entityImportance']}
    return {"data": data}


def buildEdges(relationRecord):
    data = {"id": relationRecord['r'].identity,
            "source": relationRecord['r'].start_node.identity,
            "target": relationRecord['r'].end_node.identity,
            "relationship": list(relationRecord['r'].types())[0],
            "direction":relationRecord['r']['inverse']}
    return {"data": data}

In [10]:
# add limit 50:2.35s  no limit: 6.01s

## 第一周——计算importance和diversity

In [4]:
import time
from py2neo import Graph
graph = Graph("http://localhost:7474", auth=("neo4j", "990910lhylhy"))

In [5]:
kw = 'China'
command = 'MATCH (n)-[r]->(rec)-[r1]->(rec1) WHERE n.entityName=\'' + kw + '\'WITH rec,COUNT(r1) AS gs RETURN rec,labels(rec),gs ORDER BY gs DESC'
import time
start = time.time()
ss = graph.run(command).data()
mid = time.time()
print(mid-start)
# 第一次run需要一分钟左右

9.953426361083984


In [38]:
ss

[{'rec': (_3081098:Place {entityId: 3753937, entityName: 'Toronto', entityProperty: 'name: Toronto,  name: City of Toronto,  nick: T.O.,   T-Dot,   Hogtown,   The Queen City,   The Big Smoke,   Toronto the Good,   The Six,  motto: Diversity Our Strength,  areaTotal: 6.3021E8,  areaUrban: 1.75149E9,  areaMetro: 5.90571E9,  leaderTitle: Mayor,  leaderTitle: Deputy Mayor,  leaderTitle: Council,  leaderTitle: MPs,  leaderTitle: MPPs,  foundingDate: 1793-08-27,  foundingDate: 1834-03-06,  foundingDate: 1953-01-20,  foundingDate: 1998-01-01,  elevation: 76.0,  populationTotal: 2615060,  populationDensity: 4149.5,  populationMetro: 5583064,  populationUrban: 5132794,  postalCode: M,  areaCode: 416,   647,   437,  utcOffset: -5,  utcOffset: -4', entityType: 'City'}),
  'labels(rec)': ['Place'],
  'gs': 7427},
 {'rec': (_2633977:Place {entityId: 3152520, entityName: 'Beijing', entityProperty: 'name: Beijing Municipality,  areaTotal: 1.641054E10,  areaUrban: 1.36832E9,  areaRural: 1.504222E10,  

In [13]:
importance = {}
label = {}
for i in range(len(ss)):
    importance[ss[i]['rec']]=ss[i]['gs']
    label[ss[i]['rec']]=ss[i]['labels(rec)'][0]

In [14]:
importance

{(_3081098:Place {entityId: 3753937, entityName: 'Toronto', entityProperty: 'name: Toronto,  name: City of Toronto,  nick: T.O.,   T-Dot,   Hogtown,   The Queen City,   The Big Smoke,   Toronto the Good,   The Six,  motto: Diversity Our Strength,  areaTotal: 6.3021E8,  areaUrban: 1.75149E9,  areaMetro: 5.90571E9,  leaderTitle: Mayor,  leaderTitle: Deputy Mayor,  leaderTitle: Council,  leaderTitle: MPs,  leaderTitle: MPPs,  foundingDate: 1793-08-27,  foundingDate: 1834-03-06,  foundingDate: 1953-01-20,  foundingDate: 1998-01-01,  elevation: 76.0,  populationTotal: 2615060,  populationDensity: 4149.5,  populationMetro: 5583064,  populationUrban: 5132794,  postalCode: M,  areaCode: 416,   647,   437,  utcOffset: -5,  utcOffset: -4', entityType: 'City'}): 7427,
 (_2633977:Place {entityId: 3152520, entityName: 'Beijing', entityProperty: 'name: Beijing Municipality,  areaTotal: 1.641054E10,  areaUrban: 1.36832E9,  areaRural: 1.504222E10,  leaderTitle: Party Secretary,  leaderTitle: Mayor,  l

In [15]:
label

{(_3081098:Place {entityId: 3753937, entityName: 'Toronto', entityProperty: 'name: Toronto,  name: City of Toronto,  nick: T.O.,   T-Dot,   Hogtown,   The Queen City,   The Big Smoke,   Toronto the Good,   The Six,  motto: Diversity Our Strength,  areaTotal: 6.3021E8,  areaUrban: 1.75149E9,  areaMetro: 5.90571E9,  leaderTitle: Mayor,  leaderTitle: Deputy Mayor,  leaderTitle: Council,  leaderTitle: MPs,  leaderTitle: MPPs,  foundingDate: 1793-08-27,  foundingDate: 1834-03-06,  foundingDate: 1953-01-20,  foundingDate: 1998-01-01,  elevation: 76.0,  populationTotal: 2615060,  populationDensity: 4149.5,  populationMetro: 5583064,  populationUrban: 5132794,  postalCode: M,  areaCode: 416,   647,   437,  utcOffset: -5,  utcOffset: -4', entityType: 'City'}): 'Place',
 (_2633977:Place {entityId: 3152520, entityName: 'Beijing', entityProperty: 'name: Beijing Municipality,  areaTotal: 1.641054E10,  areaUrban: 1.36832E9,  areaRural: 1.504222E10,  leaderTitle: Party Secretary,  leaderTitle: Mayor,

In [16]:
len(importance)

13690

In [17]:
len(label)

13690

In [18]:
from collections import Counter
ll = list(label.values())
nums = Counter(ll)
numvalues = list(dict(nums).values())
percentage = []
for i in range(len(numvalues)):
    percentage.append(numvalues[i]/len(ss))
percentage

[0.3358655953250548,
 0.036303871439006576,
 0.1310445580715851,
 0.40219138056975895,
 0.01241782322863404,
 0.01701972242512783,
 0.023447772096420745,
 0.03294375456537619,
 0.0075237399561723884,
 0.0006574141709276845,
 0.00014609203798392987,
 7.304601899196493e-05,
 0.0003652300949598247]

In [19]:
# 问题：
# 怎样将各label不同的percentage转换为量化数值？
# 用一个算法将所有importance和diversity量化在一个区间(如0-10)中
# 一种label个数越多，它对应的量化数字越小

In [20]:
lab_per = {}
for j in range(len(nums)):
    lab_per[list(nums.keys())[j]]=percentage[j]
lab_per

{'Place': 0.3358655953250548,
 'Thing': 0.036303871439006576,
 'Organisation': 0.1310445580715851,
 'Person': 0.40219138056975895,
 'Food': 0.01241782322863404,
 'Transportation': 0.01701972242512783,
 'Event': 0.023447772096420745,
 'Work': 0.03294375456537619,
 'Device': 0.0075237399561723884,
 'Award': 0.0006574141709276845,
 'TimePeriod': 0.00014609203798392987,
 'Activity': 7.304601899196493e-05,
 'Species': 0.0003652300949598247}

In [21]:
import numpy as np
import pandas as pd
df = pd.DataFrame(ll)

In [22]:
replaced = list(df[0].map(lab_per))
replaced

[0.3358655953250548,
 0.3358655953250548,
 0.036303871439006576,
 0.036303871439006576,
 0.3358655953250548,
 0.036303871439006576,
 0.036303871439006576,
 0.3358655953250548,
 0.036303871439006576,
 0.036303871439006576,
 0.036303871439006576,
 0.1310445580715851,
 0.3358655953250548,
 0.3358655953250548,
 0.3358655953250548,
 0.1310445580715851,
 0.1310445580715851,
 0.3358655953250548,
 0.40219138056975895,
 0.3358655953250548,
 0.3358655953250548,
 0.3358655953250548,
 0.1310445580715851,
 0.3358655953250548,
 0.40219138056975895,
 0.3358655953250548,
 0.3358655953250548,
 0.3358655953250548,
 0.40219138056975895,
 0.3358655953250548,
 0.40219138056975895,
 0.3358655953250548,
 0.3358655953250548,
 0.1310445580715851,
 0.3358655953250548,
 0.1310445580715851,
 0.1310445580715851,
 0.3358655953250548,
 0.1310445580715851,
 0.40219138056975895,
 0.1310445580715851,
 0.3358655953250548,
 0.1310445580715851,
 0.3358655953250548,
 0.1310445580715851,
 0.40219138056975895,
 0.33586559532

In [23]:
new_score = {}
for k in range(len(label)):
    new_score[list(label.keys())[k]] = replaced[k]

In [24]:
print(len(new_score))
new_score

13690


{(_3081098:Place {entityId: 3753937, entityName: 'Toronto', entityProperty: 'name: Toronto,  name: City of Toronto,  nick: T.O.,   T-Dot,   Hogtown,   The Queen City,   The Big Smoke,   Toronto the Good,   The Six,  motto: Diversity Our Strength,  areaTotal: 6.3021E8,  areaUrban: 1.75149E9,  areaMetro: 5.90571E9,  leaderTitle: Mayor,  leaderTitle: Deputy Mayor,  leaderTitle: Council,  leaderTitle: MPs,  leaderTitle: MPPs,  foundingDate: 1793-08-27,  foundingDate: 1834-03-06,  foundingDate: 1953-01-20,  foundingDate: 1998-01-01,  elevation: 76.0,  populationTotal: 2615060,  populationDensity: 4149.5,  populationMetro: 5583064,  populationUrban: 5132794,  postalCode: M,  areaCode: 416,   647,   437,  utcOffset: -5,  utcOffset: -4', entityType: 'City'}): 0.3358655953250548,
 (_2633977:Place {entityId: 3152520, entityName: 'Beijing', entityProperty: 'name: Beijing Municipality,  areaTotal: 1.641054E10,  areaUrban: 1.36832E9,  areaRural: 1.504222E10,  leaderTitle: Party Secretary,  leaderTi

In [25]:
# 两种选择方式：
# 1. 用背包算法
# 背包算法是取舍重量和价值，这里是将diversity设为重量，将importance设为价值吗？

# 2.用一个算法综合importance和diversity两个变量，给每个entity输出一个分数，选择前20或50个分数最高的节点
# 问题：importance和diversity的值域不同，是否需要normalize？

# 第二周——背包算法 First Test

In [33]:
n = len(ss)
c = 5
# w里不能有float
w = [int(x*100) for x in replaced]
v = list(importance.values())

In [34]:
w

[33,
 33,
 3,
 3,
 33,
 3,
 3,
 33,
 3,
 3,
 3,
 13,
 33,
 33,
 33,
 13,
 13,
 33,
 40,
 33,
 33,
 33,
 13,
 33,
 40,
 33,
 33,
 33,
 40,
 33,
 40,
 33,
 33,
 13,
 33,
 13,
 13,
 33,
 13,
 40,
 13,
 33,
 13,
 33,
 13,
 40,
 33,
 33,
 40,
 3,
 40,
 3,
 33,
 33,
 13,
 40,
 33,
 33,
 3,
 13,
 13,
 13,
 40,
 33,
 33,
 13,
 3,
 40,
 40,
 1,
 40,
 40,
 40,
 40,
 33,
 13,
 33,
 40,
 13,
 33,
 33,
 40,
 40,
 13,
 33,
 40,
 40,
 40,
 40,
 33,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 13,
 33,
 33,
 13,
 40,
 3,
 40,
 40,
 40,
 40,
 40,
 1,
 40,
 33,
 33,
 40,
 40,
 13,
 40,
 33,
 33,
 13,
 40,
 40,
 1,
 40,
 3,
 40,
 40,
 1,
 40,
 33,
 33,
 40,
 40,
 33,
 40,
 40,
 40,
 33,
 13,
 13,
 40,
 3,
 33,
 40,
 40,
 2,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 33,
 1,
 1,
 40,
 40,
 40,
 3,
 40,
 40,
 33,
 33,
 33,
 13,
 1,
 33,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 13,
 40,
 40,
 13,
 40,
 13,
 40,
 13,
 13,
 40,
 40,
 40,
 40,
 13,
 40,
 40,
 40,
 33,
 33,
 33,
 3,
 40,
 40,
 40,
 40,
 40,
 33,
 40,

In [35]:
def bag(n,c,w,v):
	res=[[-1 for j in range(c+1)] for i in range(n+1)]
	for j in range(c+1):
		res[0][j]=0
	for i in range(1,n+1):
		for j in range(1,c+1):
			res[i][j]=res[i-1][j]
			if j>=w[i-1] and res[i][j]<res[i-1][j-w[i-1]]+v[i-1]:
				res[i][j]=res[i-1][j-w[i-1]]+v[i-1]
	return res

In [36]:
def show(n,c,w,res):
	print('Total value:',res[n][c])
	x=[False for i in range(n)]
	j=c
	for i in range(1,n+1):
		if res[i][j]>res[i-1][j]:
			x[i-1]=True
			j-=w[i-1]
	print('Selected:')
	ccc = 0
	for i in range(n):
		if x[i]:
			print(ss[i]['rec'])
			ccc += 1
	print(ccc)

In [37]:
res=bag(n,c,w,v)
show(n,c,w,res)

Total value: 3419
Selected:
(_3061716:Thing {entityId: 3806158, entityName: 'Americans', entityProperty: 'totalPopulation: 308745538,  totalPopulation: 2010,  totalPopulation: 320925485,  totalPopulation: 2015,  name: Americans', entityType: 'EthnicGroup'})
(_899381:Food {entityId: 1015135, entityName: 'Tea', entityProperty: 'name: Tea', entityType: 'Beverage'})
(_2691391:Device {entityId: 3397046, entityName: 'Type 56 assault rifle', entityProperty: 'weight: 4030.0,  length: 0.874,  name: Chinese Norinco Type 56', entityType: 'Weapon'})
(_1663044:Award {entityId: 2065883, entityName: 'Golden Phoenix Awards', entityProperty: 'name: Golden Phoenix Awards,  year: 1987', entityType: 'Award'})
(_2335947:Device {entityId: 2992293, entityName: 'Type 69 RPG', entityProperty: 'weight: 5600.0,  length: 0.91,  name: Norinco Type 69 RPG', entityType: 'Weapon'})
(_2514432:TimePeriod {entityId: 3080807, entityName: '2003 in spaceflight', entityProperty: 'totalLaunches: 63,  successfulLaunches: 60, 